# Notebook 2.5.5: Glucose (The Minimal Model)

---

<br>

*Modeling and Simulation in Python*

Copyright 2021 Allen Downey, (License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/))

Revised, Mike Augspurger (2021-present)

<br>

---

In this notebook we'll start a new example: a model of how glucose and insulin interact to control blood sugar. We will implement a widely used model called the "minimal model" because it is intended to include only the elements essential to explain the observed behavior of the system.


## Investigating the Minimal Model for Glucose and Insulin

### The Glucose-Insulin Interaction

*Pharmacokinetics* is the study of how drugs and other substances move around the body, react, and are eliminated. This notebook presents a widely used pharmacokinetic model of glucose and insulin in the blood stream.

<br>

*Glucose* is a form of sugar that circulates in the blood of animals; it is used as fuel for muscles, the brain, and other organs. The concentration of blood sugar is controlled by the hormone system, and especially by *insulin*, which is produced by the pancreas and has the effect of reducing blood sugar.

<br>

In people with normal pancreatic function, the hormone system maintains *homeostasis*; that is, it keeps the concentration of blood sugar in a range that is neither too high or too low.  But if the pancreas does not produce enough insulin, blood sugar can
become elevated, a condition called *hyperglycemia*. Long term, severe hyperglycemia is the defining symptom of *diabetes mellitus*, a serious disease that affects almost 10% of the population in the U.S.

<br>

A widely used test for hyperglycemia and diabetes is the
frequently sampled intravenous glucose tolerance test (FSIGT), in which glucose is injected into the blood stream of a fasting subject (someone who has not eaten recently); then blood samples are collected at intervals of 2--10 minutes for 3 hours. The samples are analyzed to measure the concentrations of glucose and insulin; this analysis gives us a sense of how well the body is responding to increased glucose levels.

---

<br>

🟨 🟨 Active Reading: Matching (Submit "Definition" letters in order of "Terms", separated by a space)



In [ ]:
import pandas as pd
import numpy as np
from urllib.request import urlretrieve

location = 'https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/'
folder = 'Support_files/'
name = 'Embedded_Qs.ipynb'
local, _ = urlretrieve(location + folder + name, name)
%run /content/$name

#@title { form-width: "50%", display-mode: "form" }
home = 'https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Embedded_Qs/'
data, number = display_match('2_5_systems', home,8)
answer = "" #@param {type:"string"}
a = answer.split(sep=" ")
check_match(data,a, number)

Match the term with its description

Terms

1) Glucose
2) Insulin
3) Hyperglycemia
4) FSIGT
5) Diabetes

Definitions

A) A condition in which the sugar level in a person's blood is too high.
B) A test for diabetes during which sugar is injected into the bloodstream and sugar levels are monitored over several hours.
C) A type of sugar that circulates naturally in the human blood.
D) A long term condition characterized by severely high blood sugar levels
E) A hormone that the human body uses to reduce blood sugar levels


---

### The Glucose Minimal Model

The minimal model, as proposed by Bergman, Ider, Bowden, and
Cobelli, was developed in the 1970s. Bergman and his colleagues aimed to describe the simplest model they could that could effectively accuont for the insulin-glucose relationship found in experimental data.  

<br>

<center>
<img src = https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Images/2_5/baby.PNG width = 400>
</center>

<br>

They thought it was essential to capture two characteristics of the glucose-insulin interaction:

<br>

* When glucose is injected into the blood, glucose levels in the blood should go up quickly and then should return back to a "base" level.  This "return to normal" happens in two ways.  First glucose will naturally move out of the blood stream, and second, insulin encourages (or "catalyzes") this "glucose disposal" process.

<br>

* The effect of insulin on glucose disposal, as seen in the data, happens more slowly than we would expect if it depended primarily on the the concentration of insulin in the blood. Bergman's group hypothesized that insulin must move relatively slowly from the blood to a "remote compartment"; only when in this "compartment" can it affect glucose levels in the body.

<br>

Interestingly, at the time, the "remote compartment" was a modeling abstraction that might, or might not, represent something physical. Later, this compartment was shown to be interstitial fluid, that is, the fluid
that surrounds tissue cells.  In the history of mathematical modeling, it is common for hypothetical entities, added to models to achieve particular effects, to be found later to correspond to physical entities.  One notable example is the gene!

<br>

The implication is that to model this system and describe its *state*, we need to keep track of 3 variables:

* the concentration $G$ of glucose in the blood

* the concentration $I$ of insulin in the blood

* the concentration $X$ of insulin in the tissue fluid

In the minimal model, $I$ is a measured value that is used as an input to the model (as opposed to $G$ and $X$, which are variables).  When at high levels, the insulin in the blood will diffuse into the tissue.

---

<br>

🟨 🟨 Active Reading: Multiple Choice



In [ ]:
#@title { run: "auto", form-width: "50%", display-mode: "form" }
home = 'https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Embedded_Qs/'
data = display_multC('2_5_systems',home,9)
answer = "" # @param ["", "A", "B", "C", "D", "E"]
check_multC(data,answer)



Match the term with its description

Terms

1) Glucose
2) Insulin
3) Hyperglycemia
4) FSIGT
5) Diabetes

Definitions

A) A condition in which the sugar level in a person's blood is too high.
B) A test for diabetes during which sugar is injected into the bloodstream and sugar levels are monitored over several hours.
C) A type of sugar that circulates naturally in the human blood.
D) A long term condition characterized by severely high blood sugar levels
E) A hormone that the human body uses to reduce blood sugar levels


---

### Putting the model into differential form

Our simulation will track the state of our system ($G$ and $X$) during a FSIGT test: that is, after glucose is injected into the blood.  To do this, we will need to incorporate two differential equations into the simulation:

<br>

$$\frac{dG}{dt} = -k_1 \left[ G(t) - G_b \right] - X(t) G(t)$$

<br>

$$\frac{dX}{dt} = k_3 \left[I(t) - I_b \right] - k_2 X(t)$$

<br>

The constants in this equation all have subscripts, and include:

* $G_b$ is the base concentration of blood glucose (that is, the glucose concentrations at equilibrium. This is estimated from measurements at the beginning or end of the test).

* $I_b$ is the base concentration of blood insulin.

* $k_1$, $k_2$, and $k_3$ are positive-valued parameters that control the rates of appearance and disappearance for glucose and insulin.

We can interpret the terms in the equations one by one:

<br>

-   $-k_1 \left[ G(t) - G_b \right]$ is the rate of glucose change due to the natural self-balancing of the glucose. When $G(t)$ is above base level, $G_b$, this term is negative; when $G(t)$ is below base level this term is positive. So in the absence of insulin, this term tends to restore blood glucose to base level.

-   $-X(t) G(t)$ models the interaction of glucose and insulin in tissue fluid, so the rate increases as either $X$ or $G$ increases. This term does not require a rate parameter because the units of $X$ are unspecified.

-   $k_3 \left[ I(t) - I_b \right]$ is the rate at which insulin diffuses between blood and tissue fluid (where it can help dispose of glucose). When $I(t)$ is above base level $I_b$, insulin diffuses from the blood into the tissue fluid. When $I(t)$ is below base level, insulin diffuses from tissue to the blood.

-   $-k_2 X(t)$ is the rate of insulin disappearance in tissue fluid as it is consumed or broken down.

<br>

The initial state of the model is:

* $X(0) = I_b$, which means that the level of insulin in the tissue at equilibrium with the base level of insulin in the blood.
* $G(0) = G_0$, where $G_0$ is a constant that represents the concentration of blood sugar immediately after the injection. Since $G_0$ is difficult to measure, it is treated as a *free parameter* of the model, which means that we are free to choose it to fit the data.

While we can think about what each of these represent, it's very hard to predict how they will interact.  So in the next notebook we'll create a simulation to explore that behavior!

---

<br>

🟨 🟨 Active Reading: Matching (Submit "Definition" letters in order of "Terms", separated by a space)



In [ ]:
#@title { form-width: "50%", display-mode: "form" }
home = 'https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Embedded_Qs/'
data, number = display_match('2_5_systems', home,10)
answer = "" #@param {type:"string"}
a = answer.split(sep=" ")
check_match(data,a, number)



G' is the level of glucose in the blood at any time.  Look at the dG/dt equation, and match each term with the possible effects of the term.  Think about signs (+ and -);  G(t) and X(t) are always positive.

Terms

1) -k1 (G(t) - G_b)
2) - X(t) G(t)

Definitions

A) This term can only increase the level of glucose
B) This term can only decrease the level of glucose
C) This term can either increase or decrease the level of glucose


---

<br>

🟨 🟨 Active Reading: Matching (Submit "Definition" letters in order of "Terms", separated by a space)



In [ ]:
#@title { form-width: "50%", display-mode: "form" }
home = 'https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Embedded_Qs/'
data, number = display_match('2_5_systems', home,11)
answer = "" #@param {type:"string"}
a = answer.split(sep=" ")
check_match(data,a, number)



G' is the level of glucose in the blood at any time.  Look at the dG/dt equation, and match each term with the possible effects of the term.  Think about signs (+ and -);  G(t) and X(t) are always positive.

Terms

1) -k1 (G(t) - G_b)
2) - X(t) G(t)

Definitions

A) This term can only increase the level of glucose
B) This term can only decrease the level of glucose
C) This term can either increase or decrease the level of glucose
